[Reference](https://preettheman.medium.com/very-basic-introduction-to-machine-learning-with-keras-python-a5f5fc4a5b0b)

In [1]:
import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras

In [2]:
print(tf.__version__)

2.5.0


In [3]:
import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
  raise Exception('This notebook is compatible with TensorFlow 1.14 or   higher, for TensorFlow 1.13 or lower please use the previous version   at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/classification_iris_data_with_keras.ipynb')

In [4]:
use_tpu = True #@param {type:”boolean”}

if use_tpu:  
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:  
    TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

else:  
    TF_MASTER=''

In [5]:
tpu_address = TF_MASTER

In [6]:
epochs = 50

In [7]:
steps_per_epoch = 5

In [8]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [9]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth','PetalLength', 'PetalWidth', 'Species']

SPECIES = ['Setosa', 'Versicolor', 'Virginica']

PREDICTION_INPUT_DATA = {'SepalLength': [6.9, 5.1, 5.9, 6.0, 5.5, 6.2, 5.5, 6.3],'SepalWidth': [3.1, 3.3, 3.0, 3.4, 2.5, 2.9, 4.2, 2.8],'PetalLength': [5.4, 1.7, 4.2, 4.5, 4.0, 4.3, 1.4, 5.1],'PetalWidth': [2.1, 0.5, 1.5, 1.6, 1.3, 1.3, 0.2, 1.5],}

PREDICTION_OUTPUT_DATA = ['Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Versicolor', 'Versicolor', 'Setosa', 'Virginica']

In [15]:
def maybe_download():
  train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
  test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)
  return train_path, test_path
  
def load_data(y_name='Species'):  
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""  
    train_path, test_path = maybe_download()
    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
                                                                             'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32,   'PetalWidth': pd.np.float32, 'Species': pd.np.int32})  
    train_x, train_y = train, train.pop(y_name)
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,  'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32,  'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
  
    test_x, test_y = test, test.pop(y_name)
  
    return (train_x, train_y), (test_x, test_y)

In [16]:
def get_model():  
    return keras.Sequential([  
                             keras.layers.Dense(10, input_shape=(4,), activation=tf.nn.relu, name = "Dense_1"),  
                             keras.layers.Dense(10, activation=tf.nn.relu, name = "Dense_2"),  
                             keras.layers.Dense(3, activation=None, name = "logits"),  
                             keras.layers.Dense(3, activation=tf.nn.softmax, name = "softmax")
                             ])

In [17]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.65.156.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.65.156.26:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [18]:
with strategy.scope():
    model = get_model()
    model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1),loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
    model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
Dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
logits (Dense)               (None, 3)                 33        
_________________________________________________________________
softmax (Dense)              (None, 3)                 12        
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

# Train the model

model.fit(train_x.values, train_y.values,steps_per_epoch = steps_per_epoch,epochs=epochs,)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if sys.path[0] == '':


Epoch 1/50
5/5 [==============================] - 1s 9ms/step - loss: 2.5588 - accuracy: 0.2583
Epoch 2/50
5/5 [==============================] - 0s 10ms/step - loss: 0.8678 - accuracy: 0.6000
Epoch 3/50
5/5 [==============================] - 0s 9ms/step - loss: 0.7590 - accuracy: 0.7000
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 0.6881 - accuracy: 0.7000
Epoch 5/50
5/5 [==============================] - 0s 10ms/step - loss: 0.6410 - accuracy: 0.7000
Epoch 6/50
5/5 [==============================] - 0s 10ms/step - loss: 0.6132 - accuracy: 0.7083
Epoch 7/50
5/5 [==============================] - 0s 9ms/step - loss: 0.5750 - accuracy: 0.7167
Epoch 8/50
5/5 [==============================] - 0s 10ms/step - loss: 0.5405 - accuracy: 0.7000
Epoch 9/50
5/5 [==============================] - 0s 10ms/step - loss: 0.5416 - accuracy: 0.7417
Epoch 10/50
5/5 [==============================] - 0s 11ms/step - loss: 0.5042 - accuracy: 0.7917
Epoch 11/50
5/5 [================

In [20]:
model.evaluate(test_x.values, test_y.values,batch_size=8)

4/4 [==============================] - 2s 18ms/step - loss: 0.1206 - accuracy: 0.9667


[0.1205725446343422, 0.9666666984558105]

In [23]:
model.save_weights('./DNN_TPU_1024.h5', overwrite=True)

In [25]:
COLUMNS_NAME=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
data = pd.DataFrame(PREDICTION_INPUT_DATA, columns=COLUMNS_NAME)
print(data)

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.9         3.1          5.4         2.1
1          5.1         3.3          1.7         0.5
2          5.9         3.0          4.2         1.5
3          6.0         3.4          4.5         1.6
4          5.5         2.5          4.0         1.3
5          6.2         2.9          4.3         1.3
6          5.5         4.2          1.4         0.2
7          6.3         2.8          5.1         1.5


In [26]:
predictions = model.predict(data.values.astype(np.float32))

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(predictions, PREDICTION_OUTPUT_DATA):  
    class_index = np.argmax(pred_dict)  
    class_probability = np.max(pred_dict)
    print(template.format(SPECIES[class_index], 100*class_probability, expec))


Prediction is "Virginica" (94.0%), expected "Virginica"

Prediction is "Setosa" (97.9%), expected "Setosa"

Prediction is "Versicolor" (89.6%), expected "Versicolor"

Prediction is "Versicolor" (88.4%), expected "Versicolor"

Prediction is "Versicolor" (84.3%), expected "Versicolor"

Prediction is "Versicolor" (94.9%), expected "Versicolor"

Prediction is "Setosa" (99.2%), expected "Setosa"

Prediction is "Virginica" (63.7%), expected "Virginica"


In [27]:
cpu_model = get_model()

cpu_model.load_weights('./DNN_TPU_1024.h5')

cpu_predictions = cpu_model.predict(data)

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(cpu_predictions, PREDICTION_OUTPUT_DATA):
    class_index = np.argmax(pred_dict)
    class_probability = np.max(pred_dict)
    print(template.format(SPECIES[class_index], 100*class_probability, expec))


Prediction is "Virginica" (94.0%), expected "Virginica"

Prediction is "Setosa" (97.9%), expected "Setosa"

Prediction is "Versicolor" (89.0%), expected "Versicolor"

Prediction is "Versicolor" (88.0%), expected "Versicolor"

Prediction is "Versicolor" (83.8%), expected "Versicolor"

Prediction is "Versicolor" (94.9%), expected "Versicolor"

Prediction is "Setosa" (99.2%), expected "Setosa"

Prediction is "Virginica" (64.6%), expected "Virginica"
